# Notas Fiscais Eletrônicas da União - Portal da Transparência CGU NFe

In [1]:
#| hide: true
! rm -rf ../data/inputs/cgu_nfe/*
! rm ../data/output/cgu/*

zsh:1: no matches found: ../data/inputs/cgu_nfe/*
zsh:1: no matches found: ../data/output/cgu/*


In [2]:

from IPython.display import display
import pandas as pd
import numpy as np
from decimal import Decimal

pd.set_option("display.float_format", lambda x: f"{x:_.2f}") 
def underscore_fmt(x):
    if pd.isna(x): return ""
    if isinstance(x, (int, np.integer)):   return f"{x:_d}"
    if isinstance(x, (float, np.floating)): return f"{x:_.0f}"
    if isinstance(x, Decimal):             return f"{x:_.0f}"
    return x



## Fonte dos dados

O Portal da Transparência do Governo Federal disponibiliza dados sobre Notas Fiscais Eletrônicas (NFE) da União que podem ser baixados no endereço:

https://portaldatransparencia.gov.br/download-de-dados/notas-fiscais/

Cada mês possui um arquivo zipado que contém os dados das notas fiscais eletrônicas emitidas no período. O nome do arquivo segue o padrão `YYYYMM_NFe.zip`, onde `YYYY` é o ano e `MM` é o mês.

Dentro do Zip encontramos os seguintes arquivos em formato CSV e codificação `latin-1`:
- `YYYYMM_NFe_NotaFiscal.csv`: Contém os dados das notas fiscais eletrônicas.
- `YYYYMM_NFe_NotaFiscalItem.csv`: Contém os dados dos itens das notas fiscais eletrônicas.
- `YYYYMM_NFe_NotaFiscalEvento.csv`: Contém os dados dos eventos atribuidos às notas fiscais eletrônicas.

Para baixar os dados de um mês específico, é preciso baixar da url:

https://portaldatransparencia.gov.br/download-de-dados/notas-fiscais/[YYYYMM]


No portal da transparência, há dados de Novembro de 2019 até o mês atual com exceção do período de janeiro de 2020 até abril de 2021. Não há nada que indique se os dados do mês atual são atualizados diariamente. 



## Baixando os dados

Vamos baixar os dados 2022 a 2024. Mas primeiro, um teste.

In [3]:
import io, zipfile
from pathlib import Path
from urllib import request


def cgu_download(year, month):
    out_dir = Path("../data/inputs/cgu_nfe/"); out_dir.mkdir(parents=True, exist_ok=True)
    url = f"https://portaldatransparencia.gov.br/download-de-dados/notas-fiscais/{year}{month:02d}"
    with request.urlopen(url) as response:
        buf = io.BytesIO(response.read())
    with zipfile.ZipFile(buf) as z:
        [(out_dir / Path(n).name).write_bytes(z.read(n)) for n in z.namelist() if n.lower().endswith(".csv")]

In [4]:
cgu_download(2022, 1)

In [5]:
import os
os.listdir("../data/inputs/cgu_nfe/")

['202201_NFe_NotaFiscal.csv',
 '202201_NFe_NotaFiscalItem.csv',
 '202201_NFe_NotaFiscalEvento.csv']

Ok, um download funcionou.  Vamos baixar todos em paralelo.

In [6]:
import asyncio, nest_asyncio, tqdm.asyncio as tqdma
nest_asyncio.apply() # Jupyter notebook async compatibility


for year in range(2022, 2025): # max of 12 concurrent downloads
    tasks = [asyncio.to_thread(cgu_download, year, month) for month in range(1, 13)]
    await tqdma.tqdm.gather(*tasks, desc=f"Downloading {year}", total=len(tasks))

In [7]:
os.listdir("../data/inputs/cgu_nfe/")[:9]

['202406_NFe_NotaFiscalEvento.csv',
 '202401_NFe_NotaFiscalItem.csv',
 '202202_NFe_NotaFiscal.csv',
 '202311_NFe_NotaFiscalItem.csv',
 '202402_NFe_NotaFiscalEvento.csv',
 '202212_NFe_NotaFiscal.csv',
 '202310_NFe_NotaFiscalEvento.csv',
 '202406_NFe_NotaFiscalItem.csv',
 '202210_NFe_NotaFiscalItem.csv']

## Usando DuckDB para manipular os dados 


O primeiro passo é criar um arquivo duckdb com as extensões necessárias para manipular os dados.

In [9]:
import duckdb
def create_db(p, ext_core=("spatial","httpfs","json","excel"), ext_community=("zipfs",)):
    c = duckdb.connect(p)
    for e in ext_core: c.install_extension(e); c.load_extension(e)
    for e in ext_community: c.install_extension(e, repository="community"); c.load_extension(e)
    return c

db = create_db("../data/outputs/cgu/nfe.db")
db.execute("PRAGMA version").df()

,library_version,source_id,codename
0,v1.3.2,0b83e5d2f6,Ossivalis


Note que estamos usando versão 1.3.2 do DuckDB.

### RAW.NFE__NOTAS

### CGU.NOTAS

In [10]:
db.execute("""
CREATE SCHEMA IF NOT EXISTS raw;

CREATE OR REPLACE TABLE raw.nfe__notas AS
SELECT *
FROM read_csv(
    '../data/inputs/cgu_nfe/*Fiscal.csv',
    delim=';',
    header=true,
    encoding='latin-1',
    escape='\"',
    auto_type_candidates=['VARCHAR']
);
""").df()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,Count
0,5198392


Criamos uma tabela `raw.nfe__notas` com 5 milhões de linhas.

In [11]:
db.execute("describe raw.nfe__notas").df()

,column_name,column_type,null,key,default,extra
0,CHAVE DE ACESSO,VARCHAR,YES,None,None,None
1,MODELO,VARCHAR,YES,None,None,None
2,SÉRIE,VARCHAR,YES,None,None,None
3,NÚMERO,VARCHAR,YES,None,None,None
4,NATUREZA DA OPERAÇÃO,VARCHAR,YES,None,None,None
5,DATA EMISSÃO,VARCHAR,YES,None,None,None
6,EVENTO MAIS RECENTE,VARCHAR,YES,None,None,None
7,DATA/HORA EVENTO MAIS RECENTE,VARCHAR,YES,None,None,None
8,CPF/CNPJ Emitente,VARCHAR,YES,None,None,None
9,RAZÃO SOCIAL EMITENTE,VARCHAR,YES,None,None,None



O dicionário de dados com as colunas de cada arquivo pode ser encontrado em:

https://portaldatransparencia.gov.br/dicionario-de-dados/notas-fiscais

In [12]:
db.query("from raw.nfe__notas").df().head(3)

,CHAVE DE ACESSO,MODELO,SÉRIE,NÚMERO,NATUREZA DA OPERAÇÃO,DATA EMISSÃO,EVENTO MAIS RECENTE,DATA/HORA EVENTO MAIS RECENTE,CPF/CNPJ Emitente,RAZÃO SOCIAL EMITENTE,...,CÓDIGO ÓRGÃO DESTINATÁRIO,ÓRGÃO DESTINATÁRIO,CNPJ DESTINATÁRIO,NOME DESTINATÁRIO,UF DESTINATÁRIO,INDICADOR IE DESTINATÁRIO,DESTINO DA OPERAÇÃO,CONSUMIDOR FINAL,PRESENÇA DO COMPRADOR,VALOR NOTA FISCAL
0,42220107432517001847550030005499571000224173,55 - NF-E EMITIDA EM SUBSTITUIÇÃO AO MODELO 1 ...,3,549957,Outra saida merc./prest.serv. nao especif.,01/01/2022 00:02:22,Autorização de Uso,01/01/2022 00:05:21,07432517001847,SIMPRESS COMERCIO LOCACAO E SERVICOS LTDA,...,-1,Sem informação,00394460043975,MINISTERIO DA ECONOMIA,DF,NÃO CONTRIBUINTE,2 - OPERAÇÃO INTERESTADUAL,1 - CONSUMIDOR FINAL,0 - NÃO SE APLICA,"62,14"
1,42220107432517001847550030005499821000227345,55 - NF-E EMITIDA EM SUBSTITUIÇÃO AO MODELO 1 ...,3,549982,Outra saida merc./prest.serv. nao especif.,01/01/2022 00:03:59,Autorização de Uso,01/01/2022 00:05:11,07432517001847,SIMPRESS COMERCIO LOCACAO E SERVICOS LTDA,...,26434,"Instituto Federal de Educação, Ciência e Tecno...",10779511000298,"INSTITUTO FEDERAL DE EDUCACAO, CIENCIA E TECNO...",RJ,CONTRIBUINTE ISENTO,2 - OPERAÇÃO INTERESTADUAL,1 - CONSUMIDOR FINAL,0 - NÃO SE APLICA,"224,42"
2,42220107432517001847550030005499981000232969,55 - NF-E EMITIDA EM SUBSTITUIÇÃO AO MODELO 1 ...,3,549998,Outra saida merc./prest.serv. nao especif.,01/01/2022 00:06:51,Autorização de Uso,01/01/2022 00:08:25,07432517001847,SIMPRESS COMERCIO LOCACAO E SERVICOS LTDA,...,-1,Sem informação,00394460043975,MINISTERIO DA ECONOMIA,DF,NÃO CONTRIBUINTE,2 - OPERAÇÃO INTERESTADUAL,1 - CONSUMIDOR FINAL,0 - NÃO SE APLICA,"224,42"


In [13]:
db.execute("summarize raw.nfe__notas").df()

,column_name,column_type,min,max,approx_unique,avg,std,q25,q50,q75,count,null_percentage
0,CHAVE DE ACESSO,VARCHAR,11220100764614000140550010000060041186559649,53241292660406003134550050002955471000313933,4006031,<NA>,<NA>,<NA>,<NA>,<NA>,5198392,0.00
1,MODELO,VARCHAR,55 - NF-E EMITIDA EM SUBSTITUIÇÃO AO MODELO 1 ...,55 - NF-E EMITIDA EM SUBSTITUIÇÃO AO MODELO 1 ...,1,<NA>,<NA>,<NA>,<NA>,<NA>,5198392,0.00
2,SÉRIE,VARCHAR,0,99,574,<NA>,<NA>,<NA>,<NA>,<NA>,5198392,0.00
3,NÚMERO,VARCHAR,1,999999991,886203,<NA>,<NA>,<NA>,<NA>,<NA>,5198392,0.00
4,NATUREZA DA OPERAÇÃO,VARCHAR,#### - Material Consumo - Mudança de Destinação,|VENDA DE MERC ADQ OU REC DE TERCEIROS,49855,<NA>,<NA>,<NA>,<NA>,<NA>,5198392,0.00
5,DATA EMISSÃO,VARCHAR,01/01/2022 00:02:22,31/12/2024 23:59:59,3305590,<NA>,<NA>,<NA>,<NA>,<NA>,5198392,0.00
6,EVENTO MAIS RECENTE,VARCHAR,Autorização de Uso,Sem informação,7,<NA>,<NA>,<NA>,<NA>,<NA>,5198392,0.00
7,DATA/HORA EVENTO MAIS RECENTE,VARCHAR,01/01/1900 00:00:00,31/12/2024 22:51:18,3650805,<NA>,<NA>,<NA>,<NA>,<NA>,5198392,0.00
8,CPF/CNPJ Emitente,VARCHAR,***.000.033-**,98674708000122,125656,<NA>,<NA>,<NA>,<NA>,<NA>,5198392,0.00
9,RAZÃO SOCIAL EMITENTE,VARCHAR,'DYNAMICS CONSULTORIA EM CONSTRUCAO,ÚNICA RADIO DIGITAL LTDA,179958,<NA>,<NA>,<NA>,<NA>,<NA>,5198392,0.00


MODELO é sempre o mesmo. "DATA EMISSÃO" e "DATA/HORA EVENTO MAIS RECENTE" são do tipo `timestamp` e "VALOR NOTA FISCAL" é do tipo `decimal`.  Assumindo que nenhum valor ultrapassa 999 bilhões, podemos usar decimal(14,2).

In [14]:
db.execute("SELECT COUNT(*) AS n_rows FROM raw.nfe__notas").df()


,n_rows
0,5198392


A tabela de notas tem aproximadamente 5.2 milhões de linhas.  Mas algumas são repetidas. 

In [15]:
db.execute("""
           SELECT "CHAVE DE ACESSO", COUNT(*) AS n_rows 
           FROM raw.nfe__notas group by "CHAVE DE ACESSO" 
           order by n_rows desc""").df()

,CHAVE DE ACESSO,n_rows
0,11241222761584012833550010000725041312417082,3
1,13241202812552000200550010000117681000589544,3
2,33241236414637000171550010000183171004128190,3
3,41240718472579000150550020000121631188755015,3
4,13240723032493000147550010000000081684846307,3
...,...,...
5099820,42241012483930000122550010001825751303024060,1
5099821,25241010463704000154550010000067571924701707,1
5099822,51241019967950000207550010000643051413619501,1
5099823,24241040990509000143550010000084021355290630,1


In [16]:
db.execute("""       
    SELECT "DATA EMISSÃO", "EVENTO MAIS RECENTE", "DATA/HORA EVENTO MAIS RECENTE", "VALOR NOTA FISCAL" from raw.nfe__notas where "CHAVE DE ACESSO" = '21240706049258000169550010002625451000741910'
    """).df()

,DATA EMISSÃO,EVENTO MAIS RECENTE,DATA/HORA EVENTO MAIS RECENTE,VALOR NOTA FISCAL
0,13/07/2024 07:37:06,Autorização de Uso,13/07/2024 07:37:15,"4856,32"
1,13/07/2024 07:37:06,Autorização de Uso,13/07/2024 07:37:15,"4856,32"
2,13/07/2024 07:37:06,Autorização de Uso,13/07/2024 07:37:15,"4856,32"


In [17]:
db.execute("""       
    SELECT DISTINCT * from raw.nfe__notas where "CHAVE DE ACESSO" = '21240706049258000169550010002625451000741910'
    """).df()

,CHAVE DE ACESSO,MODELO,SÉRIE,NÚMERO,NATUREZA DA OPERAÇÃO,DATA EMISSÃO,EVENTO MAIS RECENTE,DATA/HORA EVENTO MAIS RECENTE,CPF/CNPJ Emitente,RAZÃO SOCIAL EMITENTE,...,CÓDIGO ÓRGÃO DESTINATÁRIO,ÓRGÃO DESTINATÁRIO,CNPJ DESTINATÁRIO,NOME DESTINATÁRIO,UF DESTINATÁRIO,INDICADOR IE DESTINATÁRIO,DESTINO DA OPERAÇÃO,CONSUMIDOR FINAL,PRESENÇA DO COMPRADOR,VALOR NOTA FISCAL
0,21240706049258000169550010002625451000741910,55 - NF-E EMITIDA EM SUBSTITUIÇÃO AO MODELO 1 ...,1,262545,VENDA DE PROD. PROP,13/07/2024 07:37:06,Autorização de Uso,13/07/2024 07:37:15,06049258000169,GRANORTE S/A,...,52121,Comando do Exército,07524710000160,3 BATALHAO DE ENGENHARIA DE CONSTRUCAO,PI,NÃO CONTRIBUINTE,2 - OPERAÇÃO INTERESTADUAL,1 - CONSUMIDOR FINAL,1 - OPERAÇÃO PRESENCIAL,"4856,32"


In [18]:
distinct_notas = db.execute("SELECT DISTINCT * FROM raw.nfe__notas").df()
distinct_notas.shape

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

(5099924, 25)

In [19]:
db.execute("""SELECT SUM((REPLACE("VALOR NOTA FISCAL", ',', '.')::DECIMAL(14,2))) as valor_nota FROM distinct_notas""").df()

,valor_nota
0,232_231_315_906.67


100 mil linhas, aproximadamente, são repetidas. E o valor total das notas é aproximadamente 232 bilhões de reais.

In [20]:
cols = db.execute("""
SELECT column_name
FROM (summarize raw.nfe__notas)
WHERE approx_unique < 30
""").df()["column_name"].tolist()
cols

['MODELO',
 'EVENTO MAIS RECENTE',
 'UF EMITENTE',
 'UF DESTINATÁRIO',
 'INDICADOR IE DESTINATÁRIO',
 'DESTINO DA OPERAÇÃO',
 'CONSUMIDOR FINAL',
 'PRESENÇA DO COMPRADOR']

In [21]:

qid = lambda c: '"' + c.replace('"', '""') + '"'

# build UNION of per-column distinct lists
sql = " UNION ALL ".join(
    f"""
    SELECT 
        '{c}' AS column_name, 
        len(LIST(DISTINCT {qid(c)})) AS n_distinct,
        LIST_SORT(LIST(DISTINCT {qid(c)})) AS distinct_values
    FROM raw.nfe__notas
    """
    for c in cols
)


enum_df = db.execute(sql).df()

enum_df

,column_name,n_distinct,distinct_values
0,MODELO,1,[55 - NF-E EMITIDA EM SUBSTITUIÇÃO AO MODELO 1...
1,EVENTO MAIS RECENTE,8,"[Autorização de Uso, Cancelamento da NF-e, Car..."
2,UF EMITENTE,27,"[AC, AL, AM, AP, BA, CE, DF, ES, GO, MA, MG, M..."
3,UF DESTINATÁRIO,27,"[AC, AL, AM, AP, BA, CE, DF, ES, GO, MA, MG, M..."
4,INDICADOR IE DESTINATÁRIO,3,"[CONTRIBUINTE ICMS, CONTRIBUINTE ISENTO, NÃO C..."
5,DESTINO DA OPERAÇÃO,3,"[1 - OPERAÇÃO INTERNA, 2 - OPERAÇÃO INTERESTAD..."
6,CONSUMIDOR FINAL,2,"[0 - NORMAL, 1 - CONSUMIDOR FINAL]"
7,PRESENÇA DO COMPRADOR,6,"[0 - NÃO SE APLICA, 1 - OPERAÇÃO PRESENCIAL, 2..."


In [22]:
list(enum_df.distinct_values)

[array(['55 - NF-E EMITIDA EM SUBSTITUIÇÃO AO MODELO 1 OU 1A'],
       dtype=object),
 array(['Autorização de Uso', 'Cancelamento da NF-e', 'Carta de correção',
        'Manifestação do destinatário - Ciência da operação',
        'Manifestação do destinatário - Confirmação da operação',
        'Manifestação do destinatário - Desconhecimento da operação',
        'Manifestação do destinatário - Operação não realizada',
        'Sem informação'], dtype=object),
 array(['AC', 'AL', 'AM', 'AP', 'BA', 'CE', 'DF', 'ES', 'GO', 'MA', 'MG',
        'MS', 'MT', 'PA', 'PB', 'PE', 'PI', 'PR', 'RJ', 'RN', 'RO', 'RR',
        'RS', 'SC', 'SE', 'SP', 'TO'], dtype=object),
 array(['AC', 'AL', 'AM', 'AP', 'BA', 'CE', 'DF', 'ES', 'GO', 'MA', 'MG',
        'MS', 'MT', 'PA', 'PB', 'PE', 'PI', 'PR', 'RJ', 'RN', 'RO', 'RR',
        'RS', 'SC', 'SE', 'SP', 'TO'], dtype=object),
 array(['CONTRIBUINTE ICMS', 'CONTRIBUINTE ISENTO', 'NÃO CONTRIBUINTE'],
       dtype=object),
 array(['1 - OPERAÇÃO INTERNA', '2 

In [23]:
create_enums_sql = """
DROP TYPE IF EXISTS tipo_modelo;
CREATE TYPE tipo_modelo AS ENUM ('55 - NF-E EMITIDA EM SUBSTITUIÇÃO AO MODELO 1 OU 1A');

DROP TYPE IF EXISTS tipo_evento;
CREATE TYPE tipo_evento AS ENUM (
    'Autorização de Uso', 
    'Cancelamento da NF-e', 
    'Carta de correção',
    'Manifestação do destinatário - Ciência da operação',
    'Manifestação do destinatário - Confirmação da operação',
    'Manifestação do destinatário - Desconhecimento da operação',
    'Manifestação do destinatário - Operação não realizada',
    'Sem informação'
);

DROP TYPE IF EXISTS uf;
CREATE TYPE uf AS ENUM (
    'AC', 'AL', 'AM', 'AP', 'BA', 'CE', 'DF', 'ES', 'GO',
    'MA', 'MG', 'MS', 'MT', 'PA', 'PB', 'PE', 'PI', 'PR',
    'RJ', 'RN', 'RO', 'RR', 'RS', 'SC', 'SE', 'SP', 'TO'
);

DROP TYPE IF EXISTS tipo_contribuinte;
CREATE TYPE tipo_contribuinte AS ENUM (
    'CONTRIBUINTE ICMS',
    'CONTRIBUINTE ISENTO', 
    'NÃO CONTRIBUINTE'
);

DROP TYPE IF EXISTS tipo_destino;
CREATE TYPE tipo_destino AS ENUM (
    '1 - OPERAÇÃO INTERNA',
    '2 - OPERAÇÃO INTERESTADUAL',
    '3 - OPERAÇÃO COM EXTERIOR'
);

DROP TYPE IF EXISTS tipo_consumidor;
CREATE TYPE tipo_consumidor AS ENUM (
    '0 - NORMAL',
    '1 - CONSUMIDOR FINAL'
);

DROP TYPE IF EXISTS tipo_presenca;
CREATE TYPE tipo_presenca AS ENUM (
    '0 - NÃO SE APLICA',
    '1 - OPERAÇÃO PRESENCIAL',
    '2 - OPERAÇÃO NÃO PRESENCIAL, PELA INTERNET',
    '3 - OPERAÇÃO NÃO PRESENCIAL, TELEATENDIMENTO',
    '5 - NÃO INFORMADO',
    '9 - OPERAÇÃO NÃO PRESENCIAL, OUTROS'
);
"""

In [24]:
db.execute("CREATE SCHEMA IF NOT EXISTS cgu;")
db.execute(create_enums_sql)


In [25]:
db.execute("select column_name from (describe raw.nfe__notas)").df()["column_name"].to_list()

['CHAVE DE ACESSO',
 'MODELO',
 'SÉRIE',
 'NÚMERO',
 'NATUREZA DA OPERAÇÃO',
 'DATA EMISSÃO',
 'EVENTO MAIS RECENTE',
 'DATA/HORA EVENTO MAIS RECENTE',
 'CPF/CNPJ Emitente',
 'RAZÃO SOCIAL EMITENTE',
 'INSCRIÇÃO ESTADUAL EMITENTE',
 'UF EMITENTE',
 'MUNICÍPIO EMITENTE',
 'CÓDIGO ÓRGÃO SUPERIOR DESTINATÁRIO',
 'ÓRGÃO SUPERIOR DESTINATÁRIO',
 'CÓDIGO ÓRGÃO DESTINATÁRIO',
 'ÓRGÃO DESTINATÁRIO',
 'CNPJ DESTINATÁRIO',
 'NOME DESTINATÁRIO',
 'UF DESTINATÁRIO',
 'INDICADOR IE DESTINATÁRIO',
 'DESTINO DA OPERAÇÃO',
 'CONSUMIDOR FINAL',
 'PRESENÇA DO COMPRADOR',
 'VALOR NOTA FISCAL']

In [26]:
create_cgu__notas_sql = """
SELECT DISTINCT
  "CHAVE DE ACESSO" AS chave_acesso, -- identificador único da nota fiscal eletrônica
  "MODELO"::tipo_modelo AS modelo, -- código do modelo de documento fiscal (sempre 55 - NFE)
  "SÉRIE"::USMALLINT AS serie, -- série da nota fiscal. juntamente com o número identifica unicamente a nota fiscal
  "NÚMERO"::BIGINT AS numero, -- número da nota fiscal. juntamente com a série identifica unicamente a nota fiscal
  "NATUREZA DA OPERAÇÃO" AS natureza_operacao, -- descrição da natureza da operação. campo de preenchimento livre.
  strptime("DATA EMISSÃO", '%d/%m/%Y %H:%M:%S') AS timestamp_emissao, -- data e hora de emissão da nota fiscal
  timestamp_emissao::date AS data_emissao, -- data de emissão da nota fiscal
  month(data_emissao)::INT AS mes_emissao, -- mês de emissão da nota fiscal
  year(data_emissao)::INT AS ano_emissao, -- ano de emissão da nota fiscal
  "EVENTO MAIS RECENTE"::tipo_evento AS ultimo_evento, -- evento mais recente associado a nota fiscal (indica status atual da nota)
  strptime("DATA/HORA EVENTO MAIS RECENTE", '%d/%m/%Y %H:%M:%S') AS timestamp_ultimo_evento, -- data e hora da última atualização do status da nota fiscal
  "CPF/CNPJ Emitente" AS emitente, -- cpf ou cnpj emissor da nota fiscal. fornecedor.
  "RAZÃO SOCIAL EMITENTE" AS nome_emitente, -- razão social do emissor da nota fiscal
  "INSCRIÇÃO ESTADUAL EMITENTE"::BIGINT AS inscricao_estadual_emitente, -- inscrição estadual do emissor da nota fiscal
  "UF EMITENTE"::uf AS uf_emitente, -- sigla unidade federativa do emissor da nota fiscal
  "MUNICÍPIO EMITENTE" AS municipio_emitente, -- nome município do emissor da nota fiscal. 
  "CÓDIGO ÓRGÃO SUPERIOR DESTINATÁRIO"::INT as codigo_orgao_superior_destinatario, -- código do órgão superior destinatário da nota fiscal
  "ÓRGÃO SUPERIOR DESTINATÁRIO" AS orgao_superior_destinatario, -- código do órgão superior destinatário da nota fiscal
  "CÓDIGO ÓRGÃO DESTINATÁRIO"::INT AS codigo_orgao_destinatario, -- código do órgão destinatário da nota fiscal
  "ÓRGÃO DESTINATÁRIO" AS orgao_destinatario,
  "CNPJ DESTINATÁRIO" AS destinatario, -- cnpj do destinatário da nota fiscal. cliente.
  "NOME DESTINATÁRIO" AS nome_destinatario, -- nome do destinatário da nota fiscal
  "UF DESTINATÁRIO"::uf AS uf_destinatario, -- sigla da unidade federativa do destinatário da nota fiscal
  "INDICADOR IE DESTINATÁRIO"::tipo_contribuinte AS indicador_ie_destinatario, -- indicador de inscrição estadual do destinatário da nota fiscal
  "DESTINO DA OPERAÇÃO"::tipo_destino AS destino_operacao, -- destino da operação da nota fiscal (interna, interestadual, exterior)
  "CONSUMIDOR FINAL"::tipo_consumidor AS indicador_consumidor_final, -- indica se o destinatário é consumidor final (0 - normal, 1 - consumidor final)
  "PRESENÇA DO COMPRADOR"::tipo_presenca AS indicador_presenca_comprador, -- indica se o comprador estava presente na operação (0 - não se aplica, 1 - presencial, 2 - não presencial, 3 - teleatendimento, 5 - não informado, 9 - outros)  
  REPLACE("VALOR NOTA FISCAL", ',', '.')::decimal(14,2) AS valor_nota_fiscal -- valor total da nota fiscal
FROM raw.nfe__notas order by timestamp_emissao; 
"""

In [27]:
db.execute("CREATE OR REPLACE TABLE cgu.notas AS " + create_cgu__notas_sql).df()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,Count
0,5099924


In [28]:
db.execute("from cgu.notas limit 2").df()

,chave_acesso,modelo,serie,numero,natureza_operacao,timestamp_emissao,data_emissao,mes_emissao,ano_emissao,ultimo_evento,...,codigo_orgao_destinatario,orgao_destinatario,destinatario,nome_destinatario,uf_destinatario,indicador_ie_destinatario,destino_operacao,indicador_consumidor_final,indicador_presenca_comprador,valor_nota_fiscal
0,42220107432517001847550030005499571000224173,55 - NF-E EMITIDA EM SUBSTITUIÇÃO AO MODELO 1 ...,3,549957,Outra saida merc./prest.serv. nao especif.,2022-01-01 00:02:22,2022-01-01,1,2022,Autorização de Uso,...,-1,Sem informação,00394460043975,MINISTERIO DA ECONOMIA,DF,NÃO CONTRIBUINTE,2 - OPERAÇÃO INTERESTADUAL,1 - CONSUMIDOR FINAL,0 - NÃO SE APLICA,62.14
1,42220107432517001847550030005499821000227345,55 - NF-E EMITIDA EM SUBSTITUIÇÃO AO MODELO 1 ...,3,549982,Outra saida merc./prest.serv. nao especif.,2022-01-01 00:03:59,2022-01-01,1,2022,Autorização de Uso,...,26434,"Instituto Federal de Educação, Ciência e Tecno...",10779511000298,"INSTITUTO FEDERAL DE EDUCACAO, CIENCIA E TECNO...",RJ,CONTRIBUINTE ISENTO,2 - OPERAÇÃO INTERESTADUAL,1 - CONSUMIDOR FINAL,0 - NÃO SE APLICA,224.42


In [29]:
db.execute("select sum(valor_nota_fiscal) as sum_decimal, sum(valor_nota_fiscal::float) as sum_float from cgu.notas").df()

,sum_decimal,sum_float
0,232_231_315_906.67,232_231_315_287.59


É importante usar decimal para evitar problemas de precisão com valores monetários.

#### Usando 2024 para checar os dados

In [30]:
db.execute("select sum(valor_nota_fiscal) from cgu.notas where ano_emissao = 2024").df()

,sum(valor_nota_fiscal)
0,69_649_856_206.73


Esse valor ainda não bate com o relatório do Portal da Transparência, que diz que há 1.7 milhões de notas e um valor de 72 bilhões de reais.

![](./portal_2024.png)

In [31]:
df = db.execute("FROM cgu.notas where ano_emissao = 2024").df()
df.shape

(1631584, 28)

Mas a quantidade de notas é bem próxima. Provavelmente estamos contando notas que foram canceladas. O estranho é que considerando então menos notas, o valor total deveria ser ainda menor.

In [32]:
db.execute("""
SELECT numero, serie, emitente, destinatario, COUNT(*) AS chaves
FROM cgu.notas where ano_emissao = 2024
GROUP BY ALL
HAVING chaves > 1
ORDER BY chaves DESC
""").df()

,numero,serie,emitente,destinatario,chaves
0,780,1,26461699042002,00059311001602,2
1,24529,1,21172344000158,07524768000103,2
2,58895,3,76535764002278,29979036019321,2
3,6958,1,03961467000196,00059311000711,2
4,10862,2,37262218000124,07529010000168,2
...,...,...,...,...,...
143,22206,1,05791610000174,03112386000111,2
144,3943,1,14741207000121,00394502006266,2
145,26082,1,09110430000130,29979036116289,2
146,769,1,26461699042002,00059311001602,2


In [33]:
1_631_584 - 1_631_490

94

In [34]:
dedupl = db.execute("""
SELECT *
FROM (
  SELECT
    chave_acesso, numero, serie, emitente, destinatario,
    timestamp_emissao, ultimo_evento, timestamp_ultimo_evento,
    ROW_NUMBER() OVER (
      PARTITION BY numero, serie, emitente, destinatario
      ORDER BY timestamp_emissao DESC, timestamp_ultimo_evento DESC
    ) AS rn
  FROM cgu.notas
)
WHERE rn = 1
ORDER BY numero, serie, emitente, destinatario, timestamp_emissao DESC, timestamp_ultimo_evento DESC
""").df()
dedupl

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,chave_acesso,numero,serie,emitente,destinatario,timestamp_emissao,ultimo_evento,timestamp_ultimo_evento,rn
0,33230801363700000103550000000000011150520600,1,0,01363700000103,00394502014870,2023-08-18 12:14:03,Cancelamento da NF-e,2023-08-18 12:56:02,1
1,29241102758578000137550000000000011147270400,1,0,02758578000137,10724903000845,2024-11-21 11:34:00,Autorização de Uso,2024-11-22 14:57:03,1
2,52220803380810000109550000000000011579313643,1,0,03380810000109,09593838000100,2022-08-10 13:49:25,Cancelamento da NF-e,2022-08-10 14:14:04,1
3,14230104228626000887550000000000011000518864,1,0,04228626000887,03277610000125,2023-01-06 13:25:29,Cancelamento da NF-e,2023-01-06 13:30:57,1
4,33241004576614000177550000000000011379918505,1,0,04576614000177,10952708000287,2024-10-02 15:51:59,Autorização de Uso,2024-10-02 15:52:01,1
...,...,...,...,...,...,...,...,...,...
5099664,35230414778556000118550019652997041576886939,965299704,1,14778556000118,00394460052702,2023-04-06 11:53:16,Autorização de Uso,2023-04-06 11:53:17,1
5099665,35230414778556000118550019696000371129848732,969600037,1,14778556000118,00394460007669,2023-04-19 13:39:37,Cancelamento da NF-e,2023-04-20 10:36:19,1
5099666,35230414778556000118550019696000381168763090,969600038,1,14778556000118,00394460007669,2023-04-20 10:38:30,Autorização de Uso,2023-04-20 10:38:30,1
5099667,35231214748770000121550019885750361486643650,988575036,1,14748770000121,00394460004805,2023-12-02 10:52:09,Autorização de Uso,2023-12-02 10:52:09,1


In [35]:
dedupl_sql = """
ALTER TABLE cgu.notas
ADD COLUMN IF NOT EXISTS duplicada BOOLEAN DEFAULT FALSE;

UPDATE cgu.notas AS n
SET duplicada = TRUE
FROM (
  SELECT
    chave_acesso,
    ROW_NUMBER() OVER (
      PARTITION BY numero, serie, emitente, destinatario
      ORDER BY timestamp_emissao DESC, timestamp_ultimo_evento DESC
    ) AS rn
  FROM cgu.notas
) d
WHERE n.chave_acesso = d.chave_acesso
  AND d.rn > 1;
"""
db.execute(dedupl_sql)


In [36]:
db.query("SELECT COUNT(*) AS valid_rows FROM cgu.notas WHERE duplicada=FALSE AND ano_emissao = 2024").df()

,valid_rows
0,1631342


In [37]:
db.query("SELECT sum(valor_nota_fiscal) FROM cgu.notas WHERE duplicada=FALSE AND ano_emissao = 2024").df()

,sum(valor_nota_fiscal)
0,69_644_426_013.96


Agora há 1_631_342 notas válidas para 2024, com um total de aproximadamente 70 bilhões de reais.
Um pouco diferente do que o Portal da Transparência diz, mas bem próximo (1_631_490 e 72 bilhões de reais).
Vamos ver se os valores fazem sentido.

In [38]:
db.query("SELECT uf_destinatario, sum(valor_nota_fiscal) FROM cgu.notas WHERE uf_destinatario in ('PB', 'SP') and duplicada=FALSE AND ano_emissao = 2024 group by ALL").df()

,uf_destinatario,sum(valor_nota_fiscal)
0,PB,180_117_854.77
1,SP,15_021_847_382.52


In [39]:
db.query("SELECT ultimo_evento, count(*) as n  FROM cgu.notas WHERE duplicada=FALSE AND ano_emissao = 2024 group by all").df()

,ultimo_evento,n
0,Autorização de Uso,1631335
1,Sem informação,7


In [40]:
db.execute("""
SELECT chave_acesso, numero, serie, emitente, timestamp_emissao, ultimo_evento, timestamp_ultimo_evento, valor_nota_fiscal
FROM cgu.notas
WHERE ultimo_evento = 'Cancelamento da NF-e'
""").df()

,chave_acesso,numero,serie,emitente,timestamp_emissao,ultimo_evento,timestamp_ultimo_evento,valor_nota_fiscal
0,26220130201472000173550010000023361565771305,2336,1,30201472000173,2022-01-02 23:55:00,Cancelamento da NF-e,2022-01-02 23:57:44,105.00
1,31220101095360000187550010000096561001229588,9656,1,01095360000187,2022-01-03 00:00:00,Cancelamento da NF-e,2022-01-03 14:25:23,4_800.00
2,43220106937686000128550010000150261575610257,15026,1,06937686000128,2022-01-03 00:00:00,Cancelamento da NF-e,2022-01-03 16:59:43,1_986.90
3,53220116650774000106550010000003661000502306,366,1,16650774000106,2022-01-03 00:00:00,Cancelamento da NF-e,2022-01-03 18:30:11,4_669.16
4,33220111909283000105550011111706611111170662,111170661,1,11909283000105,2022-01-03 00:00:00,Cancelamento da NF-e,2022-01-03 10:03:00,176.00
...,...,...,...,...,...,...,...,...
109104,33230916598015000133550010000038481275678440,3848,1,16598015000133,2023-09-06 13:24:53,Cancelamento da NF-e,2023-09-06 13:25:51,42_207.60
109105,23230906921660000191550010000034121720943854,3412,1,06921660000191,2023-09-06 13:32:51,Cancelamento da NF-e,2023-09-22 13:55:11,995.26
109106,41230978393592000146558950041359461120953326,4135946,895,46613430000180,2023-09-06 13:36:32,Cancelamento da NF-e,2023-09-12 11:03:40,425.00
109107,33230933735949000199550010000059311309563601,5931,1,33735949000199,2023-09-06 13:52:04,Cancelamento da NF-e,2023-09-06 14:00:46,19_404.00


In [41]:
db.execute("""
SELECT *
FROM cgu.notas where ultimo_evento = 'Cancelamento da NF-e'
and ano_emissao = 2024
""").df()

,chave_acesso,modelo,serie,numero,natureza_operacao,timestamp_emissao,data_emissao,mes_emissao,ano_emissao,ultimo_evento,...,orgao_destinatario,destinatario,nome_destinatario,uf_destinatario,indicador_ie_destinatario,destino_operacao,indicador_consumidor_final,indicador_presenca_comprador,valor_nota_fiscal,duplicada


Só há cancelamentos em 2022 e 2023. Em 2024 não há cancelamentos.

In [42]:
db.execute("""
SELECT chave_acesso, numero, serie, emitente, timestamp_emissao, ultimo_evento, timestamp_ultimo_evento, valor_nota_fiscal
FROM cgu.notas
WHERE (numero, serie, emitente) IN (
  SELECT numero, serie, emitente
  FROM cgu.notas
  GROUP BY ALL
  HAVING COUNT(*) > 1
)
ORDER BY numero asc, serie asc, emitente asc, timestamp_emissao desc
""").df()

,chave_acesso,numero,serie,emitente,timestamp_emissao,ultimo_evento,timestamp_ultimo_evento,valor_nota_fiscal
0,53241024583483000162550010000000011836332082,1,1,24583483000162,2024-10-15 13:15:14,Autorização de Uso,2024-10-15 13:15:15,32_850.00
1,33231124583483000162550010000000011104586654,1,1,24583483000162,2023-11-08 14:20:00,Autorização de Uso,2023-11-08 14:43:11,1_967.20
2,14230529369516000351550010000000011100900005,1,1,29369516000351,2023-05-10 16:01:00,Cancelamento da NF-e,2023-05-11 10:33:44,795_640.90
3,50220529369516000351550010000000011000247304,1,1,29369516000351,2022-05-25 15:27:00,Autorização de Uso,2022-05-25 16:05:08,85.17
4,53230741329639000100550010000000011837171878,1,1,41329639000100,2023-07-13 13:43:48,Autorização de Uso,2023-07-13 13:43:51,712.99
...,...,...,...,...,...,...,...,...
739,52240722962737000128550010015099301085443504,1509930,1,22962737000128,2024-07-13 09:59:13,Autorização de Uso,2024-07-13 09:59:58,2_800.00
740,31240605017780000287550030140587596236174793,14058759,3,05017780000287,2024-06-27 07:48:27,Autorização de Uso,2024-06-27 08:32:16,17.57
741,31240605017780000287550030140587591236174792,14058759,3,05017780000287,2024-06-27 07:48:27,Autorização de Uso,2024-06-27 07:54:22,17.57
742,31220816907746000113558900343841971385958773,34384197,890,***.805.416-**,2022-08-08 15:35:51,Autorização de Uso,2022-08-08 15:36:04,202.98


In [43]:
db.execute("""
SELECT chave_acesso, numero, serie, emitente, destinatario, timestamp_emissao,ultimo_evento, valor_nota_fiscal
FROM cgu.notas
WHERE (numero, serie, emitente, destinatario) IN (
    SELECT numero, serie, emitente, destinatario
    FROM cgu.notas where ano_emissao = 2024
    GROUP BY ALL
    HAVING COUNT(*) > 1
    
)
ORDER BY numero asc, serie asc, emitente asc, timestamp_emissao desc
""").df()

,chave_acesso,numero,serie,emitente,destinatario,timestamp_emissao,ultimo_evento,valor_nota_fiscal
0,32240546281449000176550010000000031776364480,3,1,46281449000176,10838653001170,2024-05-06 09:29:00,Autorização de Uso,38_472.53
1,32240546281449000176550010000000036776364481,3,1,46281449000176,10838653001170,2024-05-06 09:29:00,Autorização de Uso,38_472.53
2,13240134574086000188550550000000911142479358,91,55,34574086000188,00059311001602,2024-01-23 12:03:00,Autorização de Uso,14_276.00
3,13240134574086000188550550000000911142479358,91,55,34574086000188,00059311001602,2024-01-23 12:03:00,Autorização de Uso,14_276.00
4,14240140951262000156550010000000961439857208,96,1,40951262000156,00059311001602,2024-01-17 09:31:32,Autorização de Uso,3_300.00
...,...,...,...,...,...,...,...,...
291,52240954305743001170550100006872451765084463,687245,10,54305743001170,00059311000711,2024-09-27 17:47:24,Autorização de Uso,202_900.00
292,52240722962737000128550010015099301085443504,1509930,1,22962737000128,07529010000168,2024-07-13 09:59:13,Autorização de Uso,2_800.00
293,52240722962737000128550010015099301085443504,1509930,1,22962737000128,07529010000168,2024-07-13 09:59:13,Autorização de Uso,2_800.00
294,31240605017780000287550030140587596236174793,14058759,3,05017780000287,00396895006247,2024-06-27 07:48:27,Autorização de Uso,17.57


#### Usando 2023

![](./portal2023.png)

In [44]:
display(db.execute("""
SELECT count(*)
FROM cgu.notas 
where ano_emissao = 2023
""").df().style.format(underscore_fmt))

,count_star()
0,1_761_915


In [45]:
db.execute("""
SELECT sum(valor_nota_fiscal) as valor_total
FROM cgu.notas 
where ano_emissao = 2023
""").df()

,valor_total
0,76_231_055_839.73


In [46]:
display(db.execute("""
SELECT count(*)
FROM cgu.notas 
where ano_emissao = 2023 and ultimo_evento != 'Cancelamento da NF-e'
""").df().style.format(underscore_fmt))

,count_star()
0,1_717_659


In [47]:
db.execute("""
SELECT sum(valor_nota_fiscal) as valor_total
FROM cgu.notas 
where ano_emissao = 2023 and ultimo_evento != 'Cancelamento da NF-e'
""").df()

,valor_total
0,72_666_041_265.70


In [48]:
display(db.execute("""
SELECT count(*)
FROM cgu.notas 
where ano_emissao = 2023 and duplicada=FALSE
""").df().style.format(underscore_fmt))

,count_star()
0,1_761_826


In [49]:
display(db.execute("""
SELECT count(*)
FROM cgu.notas 
where ano_emissao = 2023 and duplicada=FALSE and ultimo_evento != 'Cancelamento da NF-e'
""").df().style.format(underscore_fmt))

,count_star()
0,1_717_570


In [50]:
db.execute("""
SELECT sum(valor_nota_fiscal) as valor_total
FROM cgu.notas 
where ano_emissao = 2023 and duplicada=FALSE and ultimo_evento != 'Cancelamento da NF-e'
""").df()

,valor_total
0,72_665_251_723.25


#### Checando participação da Saúde

In [51]:
db.execute(""" SELECT distinct orgao_superior_destinatario, codigo_orgao_superior_destinatario FROM cgu.notas order by codigo_orgao_superior_destinatario""").df()

,orgao_superior_destinatario,codigo_orgao_superior_destinatario
0,Sem informação,-1
1,Presidência da República,20000
2,Ministério do Planejamento e Orçamento,20113
3,Ministério da Agricultura e Pecuária,22000
4,"Ministério da Ciência, Tecnologia e Inovação",24000
5,Ministério da Fazenda,25000
6,Banco Central do Brasil - Orçamento Fiscal e S...,25201
7,Ministério da Educação,26000
8,"Ministério do Desenvolvimento, Indústria, Comé...",28000
9,Ministério da Justiça e Segurança Pública,30000


In [52]:
db.execute("""
SELECT ano_emissao, sum(valor_nota_fiscal) as valor_total
FROM cgu.notas 
where duplicada=FALSE and ultimo_evento != 'Cancelamento da NF-e' and codigo_orgao_superior_destinatario = 36000
group by ano_emissao
""").df()

,ano_emissao,valor_total
0,2022,38_167_450_188.47
1,2023,35_370_442_653.84
2,2024,37_128_867_641.50


In [53]:
db.execute("""
SELECT 
    ano_emissao,
    SUM(CASE WHEN codigo_orgao_superior_destinatario = 36000 THEN valor_nota_fiscal ELSE 0 END) AS valor_saude,
    SUM(valor_nota_fiscal) AS valor_total,
    100.0 * SUM(CASE WHEN codigo_orgao_superior_destinatario = 36000 THEN valor_nota_fiscal ELSE 0 END) 
          / SUM(valor_nota_fiscal) AS pct
FROM cgu.notas 
WHERE duplicada = FALSE 
  AND ultimo_evento != 'Cancelamento da NF-e'
GROUP BY ano_emissao
ORDER BY ano_emissao
""").df()

,ano_emissao,valor_saude,valor_total,pct
0,2022,38_167_450_188.47,76_234_271_827.44,50.07
1,2023,35_370_442_653.84,72_665_251_723.25,48.68
2,2024,37_128_867_641.50,69_644_426_013.96,53.31


### RAW.NFE__ITENS

In [54]:
db.execute("""
CREATE OR REPLACE TABLE raw.nfe__itens AS
SELECT *
FROM read_csv(
    '../data/inputs/cgu_nfe/*Item.csv',
    delim=';',
    header=true,
    encoding='latin-1',
    escape='\"',
    auto_type_candidates=['VARCHAR']
);
""").df()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,Count
0,18610321


In [55]:
num_itens = db.execute(""" SELECT COUNT(*) FROM raw.nfe__itens """).df()
num_itens.style.format(underscore_fmt)


,count_star()
0,18_610_321


In [56]:
db.execute("DESCRIBE raw.nfe__itens").df()

,column_name,column_type,null,key,default,extra
0,CHAVE DE ACESSO,VARCHAR,YES,None,None,None
1,MODELO,VARCHAR,YES,None,None,None
2,SÉRIE,VARCHAR,YES,None,None,None
3,NÚMERO,VARCHAR,YES,None,None,None
4,NATUREZA DA OPERAÇÃO,VARCHAR,YES,None,None,None
5,DATA EMISSÃO,VARCHAR,YES,None,None,None
6,CPF/CNPJ Emitente,VARCHAR,YES,None,None,None
7,RAZÃO SOCIAL EMITENTE,VARCHAR,YES,None,None,None
8,INSCRIÇÃO ESTADUAL EMITENTE,VARCHAR,YES,None,None,None
9,UF EMITENTE,VARCHAR,YES,None,None,None


In [57]:
db.query("SELECT sum(valor_nota_fiscal) FROM cgu.notas WHERE duplicada=FALSE AND ano_emissao = 2024").df()

,sum(valor_nota_fiscal)
0,69_644_426_013.96


In [58]:
db.execute(""" SELECT sum((REPLACE("VALOR TOTAL", ',', '.')::decimal(14,2))) as valor_total FROM raw.nfe__itens where "CHAVE DE ACESSO" in (select chave_acesso from cgu.notas where ano_emissao = 2024 and duplicada = FALSE)""").df()

,valor_total
0,80_910_949_956.16


Valores deveriam bater

In [59]:
db.execute(""" SELECT sum((REPLACE("VALOR TOTAL", ',', '.')::decimal(14,2))) as valor_total FROM raw.nfe__itens where "CHAVE DE ACESSO" in (select chave_acesso from cgu.notas where ano_emissao = 2023 and duplicada = FALSE and ultimo_evento!= 'Cancelamento da NF-e')""").df()

,valor_total
0,82_234_912_130.59


In [60]:
db.execute("""SELECT "CHAVE DE ACESSO" from raw.nfe__itens where "CHAVE DE ACESSO" not in (select chave_acesso from cgu.notas)""").df()

,CHAVE DE ACESSO


Ok, todas as chaves de acesso da tabela de itens estão na tabela de notas.

### CGU.ITENS

In [61]:
db.execute("select column_name from (describe raw.nfe__itens)").df()["column_name"].to_list()

['CHAVE DE ACESSO',
 'MODELO',
 'SÉRIE',
 'NÚMERO',
 'NATUREZA DA OPERAÇÃO',
 'DATA EMISSÃO',
 'CPF/CNPJ Emitente',
 'RAZÃO SOCIAL EMITENTE',
 'INSCRIÇÃO ESTADUAL EMITENTE',
 'UF EMITENTE',
 'MUNICÍPIO EMITENTE',
 'CÓDIGO ÓRGÃO SUPERIOR DESTINATÁRIO',
 'ÓRGÃO SUPERIOR DESTINATÁRIO',
 'CÓDIGO ÓRGÃO DESTINATÁRIO',
 'ÓRGÃO DESTINATÁRIO',
 'CNPJ DESTINATÁRIO',
 'NOME DESTINATÁRIO',
 'UF DESTINATÁRIO',
 'INDICADOR IE DESTINATÁRIO',
 'DESTINO DA OPERAÇÃO',
 'CONSUMIDOR FINAL',
 'PRESENÇA DO COMPRADOR',
 'NÚMERO PRODUTO',
 'DESCRIÇÃO DO PRODUTO/SERVIÇO',
 'CÓDIGO NCM/SH',
 'NCM/SH (TIPO DE PRODUTO)',
 'CFOP',
 'QUANTIDADE',
 'UNIDADE',
 'VALOR UNITÁRIO',
 'VALOR TOTAL']

In [62]:
itens_sql = """
SELECT DISTINCT
  "CHAVE DE ACESSO" AS chave_acesso,
  strptime("DATA EMISSÃO", '%d/%m/%Y %H:%M:%S') AS timestamp_emissao,
  timestamp_emissao::date AS data_emissao, -- data de emissão da nota fiscal
  month(data_emissao)::INT AS mes_emissao, -- mês de emissão da nota fiscal
  year(data_emissao)::INT AS ano_emissao, -- ano de emissão da nota fiscal
  "UF EMITENTE"::uf AS uf_emitente, -- sigla unidade federativa do emissor da nota fiscal
  "UF DESTINATÁRIO"::uf AS uf_destinatario, -- sigla da unidade federativa do destinatário da nota fiscal
  "CPF/CNPJ Emitente" AS emitente, -- cpf ou cnpj emissor da nota fiscal. fornecedor.
  "RAZÃO SOCIAL EMITENTE" AS nome_emitente, -- razão social do emissor da nota fiscal
  "CNPJ DESTINATÁRIO" AS destinatario, -- cnpj do destinatário da nota fiscal. cliente.
  "NOME DESTINATÁRIO" AS nome_destinatario, -- nome do destinat
  "CÓDIGO ÓRGÃO DESTINATÁRIO"::INT AS codigo_orgao_destinatario, -- código do órgão destinatário da nota fiscal
  "CÓDIGO ÓRGÃO SUPERIOR DESTINATÁRIO"::INT AS codigo_orgao_superior_destinatario, -- código do órgão superior destinatário da nota fiscal
  "NÚMERO PRODUTO"::SMALLINT AS numero_produto,
  "DESCRIÇÃO DO PRODUTO/SERVIÇO" AS descricao_produto,
  "CÓDIGO NCM/SH" AS ncm,
  "NCM/SH (TIPO DE PRODUTO)" AS descricao_ncm,
  "CFOP"::SMALLINT AS cfop,
  REPLACE("QUANTIDADE", ',', '.')::DECIMAL(14,2) AS quantidade,
  "UNIDADE" AS unidade,
  REPLACE("VALOR UNITÁRIO", ',', '.')::DECIMAL(14,2) AS valor_unitario,
  REPLACE("VALOR TOTAL", ',', '.')::DECIMAL(14,2) AS valor_item
FROM raw.nfe__itens order by chave_acesso, numero_produto
"""

db.execute(f"CREATE OR REPLACE TABLE cgu.itens AS {itens_sql}").df()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,Count
0,18510776


In [63]:
db.execute(""" FROM cgu.itens limit 3""").df()

,chave_acesso,timestamp_emissao,data_emissao,mes_emissao,ano_emissao,uf_emitente,uf_destinatario,emitente,nome_emitente,destinatario,...,codigo_orgao_superior_destinatario,numero_produto,descricao_produto,ncm,descricao_ncm,cfop,quantidade,unidade,valor_unitario,valor_item
0,11220100764614000140550010000060041186559649,2022-01-13 16:34:10,2022-01-13,1,2022,RO,RO,00764614000140,VALTAIR LEMOS LOPES LTDA EPP,07556867000177,...,-1,1,CURVA 45' 20 MM SOLDAVEL------------------,39174090,"Outros acessórios para tubos, de plásticos",5102,6.00,UNIDAD,2.75,16.50
1,11220100764614000140550010000060041186559649,2022-01-13 16:34:10,2022-01-13,1,2022,RO,RO,00764614000140,VALTAIR LEMOS LOPES LTDA EPP,07556867000177,...,-1,2,JOELHO 90 ESGOTO DN 150MM,39174090,"Outros acessórios para tubos, de plásticos",5102,6.00,UNIDAD,30.75,184.50
2,11220100764614000140550010000060041186559649,2022-01-13 16:34:10,2022-01-13,1,2022,RO,RO,00764614000140,VALTAIR LEMOS LOPES LTDA EPP,07556867000177,...,-1,3,BUCHA DE REDUCAO.SOL.CURTA 32MM X 25MM,39174090,"Outros acessórios para tubos, de plásticos",5102,6.00,UNIDAD,0.85,5.10


In [64]:
db.execute("""SELECT ano_emissao, sum(valor_item) FROM cgu.itens group by ano_emissao order by ano_emissao""").df()

,ano_emissao,sum(valor_item)
0,2022,90_536_105_091.48
1,2023,86_909_229_737.13
2,2024,80_337_907_202.54


### RAW.NFE__EVENTOS

In [65]:
db.execute("""
CREATE OR REPLACE TABLE raw.nfe__eventos AS
SELECT *
FROM read_csv(
    '../data/inputs/cgu_nfe/*Evento.csv',
    delim=';',
    header=true,
    encoding='latin-1',
    escape='\"',
    auto_type_candidates=['VARCHAR']
);
""").df()

,Count
0,250204


In [66]:
num_eventos = db.execute(""" SELECT COUNT(*) FROM raw.nfe__eventos """).df()
num_eventos.style.format(underscore_fmt)

,count_star()
0,250_204


In [67]:
num_eventos = db.execute(""" SELECT COUNT(*) FROM (SELECT DISTINCT * FROM raw.nfe__eventos) AS subquery """).df()
num_eventos.style.format(underscore_fmt)

,count_star()
0,250_064


In [68]:
db.execute("summarize raw.nfe__eventos").df()

,column_name,column_type,min,max,approx_unique,avg,std,q25,q50,q75,count,null_percentage
0,CHAVE DE ACESSO,VARCHAR,11220103552842000144550000007133711021401139,53230950506565000113550010000000061265180097,198244,<NA>,<NA>,<NA>,<NA>,<NA>,250204,0.00
1,MODELO,VARCHAR,55 - NF-E EMITIDA EM SUBSTITUIÇÃO AO MODELO 1 ...,55 - NF-E EMITIDA EM SUBSTITUIÇÃO AO MODELO 1 ...,1,<NA>,<NA>,<NA>,<NA>,<NA>,250204,0.00
2,SÉRIE,VARCHAR,0,99,282,<NA>,<NA>,<NA>,<NA>,<NA>,250204,0.00
3,NÚMERO,VARCHAR,1,99999999,84631,<NA>,<NA>,<NA>,<NA>,<NA>,250204,0.00
4,NATUREZA DA OPERAÇÃO,VARCHAR,',|VENDA DE MERC ADQ OU REC DE TERCEIROS,12223,<NA>,<NA>,<NA>,<NA>,<NA>,250204,0.00
5,DATA EMISSÃO,VARCHAR,01/01/2023 05:02:53,31/12/2022 21:04:56,220479,<NA>,<NA>,<NA>,<NA>,<NA>,250204,0.00
6,EVENTO,VARCHAR,Cancelamento da NF-e,Manifestação do destinatário - Operação não re...,5,<NA>,<NA>,<NA>,<NA>,<NA>,250204,0.00
7,DATA/HORA EVENTO,VARCHAR,01/01/2023 06:36:00,31/12/2024 13:06:39,233577,<NA>,<NA>,<NA>,<NA>,<NA>,250204,0.00
8,DESCRIÇÃO EVENTO,VARCHAR,Protocolo: 113221947980588,Protocolo: 891239662250564,280834,<NA>,<NA>,<NA>,<NA>,<NA>,250204,0.00
9,MOTIVO EVENTO,VARCHAR,###########ERRO,é nf de reposição,35194,<NA>,<NA>,<NA>,<NA>,<NA>,250204,16.38


In [69]:
eventos_sql = """
SELECT DISTINCT
    "CHAVE DE ACESSO" AS chave_acesso, -- identificador da nota fiscal em que este evento foi registrado. juntamente com timestamp_evento, forma a chave primária
    "SÉRIE" AS serie, -- série da nota fiscal. juntamente com o número identifica unicamente a nota fiscal de um emitente
    "NÚMERO" AS numero, -- número da nota fiscal. juntamente com a série identifica unicamente a nota fiscal de um emitente
    "NATUREZA DA OPERAÇÃO" AS natureza_operacao, -- descrição da natureza da operação. campo de preenchimento livre.
    strptime("DATA EMISSÃO", '%d/%m/%Y %H:%M:%S') AS timestamp_emissao, -- data e hora de emissão da nota fiscal
    "EVENTO" AS evento, -- tipo de evento registrado (autorização de uso, cancelamento, etc)
    strptime("DATA/HORA EVENTO", '%d/%m/%Y %H:%M:%S') AS timestamp_evento, -- data e hora do evento
    "DESCRIÇÃO EVENTO" AS descricao_evento, -- descrição do evento (Protocolo)
    "MOTIVO EVENTO" AS motivo_evento, -- motivo do evento. campo de preenchimento livre.
FROM raw.nfe__eventos order by chave_acesso, timestamp_evento desc; 
"""
num_eventos = db.execute("CREATE TABLE IF NOT EXISTS cgu.eventos AS " + eventos_sql).df()
num_eventos.style.format(underscore_fmt)

,Count


In [70]:
eventos = db.execute("""SELECT evento, count(*) qtde FROM cgu.eventos GROUP BY evento""").df()
eventos

,evento,qtde
0,Manifestação do destinatário - Ciência da oper...,37097
1,Carta de correção,99792
2,Manifestação do destinatário - Confirmação da ...,3861
3,Manifestação do destinatário - Desconhecimento...,107
4,Manifestação do destinatário - Operação não re...,37
5,Cancelamento da NF-e,109170


In [71]:
notas_canceladas = db.execute("""SELECT DISTINCT chave_acesso FROM cgu.notas WHERE ultimo_evento = 'Cancelamento da NF-e'""").df()
cancelamentos = db.execute("""SELECT DISTINCT chave_acesso FROM cgu.eventos WHERE evento = 'Cancelamento da NF-e'""").df()
db.execute("""select chave_acesso from cancelamentos where chave_acesso not in (select chave_acesso from notas_canceladas)""").df()

,chave_acesso
0,31220516907746000113558900332439691985170438
1,33230631959959000182550020000426051236631905
2,33230536461952000150550010000404991000162251
3,35220748600191000159550010000517911298457676
4,41221278393592000146558950030501941102992292
...,...
56,43221207752236000123550010008659901279797715
57,52230909049833000111550010000044981202309040
58,53221204799835000104550010000030681696046831
59,53221233989468000100550010000017361221213559


![](./nota_cancelada.png)

In [72]:
db.execute("SELECT chave_acesso, timestamp_emissao, ultimo_evento, timestamp_ultimo_evento, valor_nota_fiscal FROM cgu.notas where chave_acesso = '35220748600191000159550010000517911298457676'").df()

,chave_acesso,timestamp_emissao,ultimo_evento,timestamp_ultimo_evento,valor_nota_fiscal
0,35220748600191000159550010000517911298457676,2022-07-28 10:31:33,Carta de correção,2022-07-28 10:37:36,102_539.27


O último evento da nota fiscal não condiz com o que a tabela de eventos diz.

In [73]:
db.execute("SELECT chave_acesso, evento, timestamp_emissao, timestamp_evento, descricao_evento FROM cgu.eventos where chave_acesso = '35220748600191000159550010000517911298457676'").df()

,chave_acesso,evento,timestamp_emissao,timestamp_evento,descricao_evento
0,35220748600191000159550010000517911298457676,Carta de correção,2022-07-28 10:31:33,2022-07-28 10:37:36,Protocolo: 135221001493724
1,35220748600191000159550010000517911298457676,Cancelamento da NF-e,2022-07-28 10:31:33,2022-07-28 10:34:01,Protocolo: 135221001577517


As informações acima não batem com o que está no site da receita. O Protocolo: 135221001577517 está em horário diferente no site da receita.

In [74]:
db.execute("SELECT chave_acesso, count(*) as n FROM cgu.eventos where chave_acesso in (select chave_acesso from cancelamentos where chave_acesso not in (select chave_acesso from notas_canceladas)) group by all having count(distinct evento) > 1 order by n desc").df()

,chave_acesso,n
0,31230331503963000131550010000059701739577683,4
1,35220667423152000178550000004091671177038980,3
2,35230460855863000172550010000128901431841966,3
3,31230313545241000168550010000419861054917409,3
4,33220206105362000123550010005255431125404329,3
...,...,...
56,31220516907746000113558900332439691985170438,2
57,31230506335227000174550040000176451622722754,2
58,53221005439064000107550030000008001529224567,2
59,53221103635901000148550020000215331000040430,2


In [75]:
db.execute("SELECT chave_acesso, evento, timestamp_emissao, timestamp_evento, descricao_evento FROM cgu.eventos where chave_acesso = '31230331503963000131550010000059701739577683'").df()

,chave_acesso,evento,timestamp_emissao,timestamp_evento,descricao_evento
0,31230331503963000131550010000059701739577683,Manifestação do destinatário - Operação não re...,2023-03-27 12:48:09,2023-04-25 09:57:55,Protocolo: 891232652474268
1,31230331503963000131550010000059701739577683,Manifestação do destinatário - Confirmação da ...,2023-03-27 12:48:09,2023-04-10 10:32:46,Protocolo: 891232287211306
2,31230331503963000131550010000059701739577683,Manifestação do destinatário - Ciência da oper...,2023-03-27 12:48:09,2023-03-27 17:25:18,Protocolo: 891231966876233
3,31230331503963000131550010000059701739577683,Cancelamento da NF-e,2023-03-27 12:48:09,2023-03-27 12:49:37,Protocolo: 131235341908648


![](./compra_cruz.png)

## Exportando os dados

In [76]:
db.close()